In [1]:
# this is a dungeon map generator trained using a Q-learning algorithm

import numpy as np
import random

In [2]:
"""
The Cell class represents a cell located at a given position in the dungeon
Its wall type is one of 16 possible configurations for walls as given in the Dungeon.conversion dictionary
"""
class Cell:
    def __init__(self,wall_type):
        
        self.wall_type = wall_type
        # cells are initialized as not being keypoints
        self.is_entrance = False
        self.is_treasure = False
        self.is_exit = False
        
        # below a list of wall types featuring a specific wall
        self.south_walls = [3,6,7,9,11,12,13]
        self.west_walls = [4,7,8,10,12,13,14]
        self.north_walls = [1,5,8,9,11,13,14]
        self.east_walls = [2,5,6,10,11,12,14]
        
    """
    The functions below determine whether a given cell has a certain wall based on their wall_type
    """
    def has_west_wall(self):
        if self.wall_type in self.west_walls:
            return True
        else: 
            return False
    def has_north_wall(self):
        if self.wall_type in self.north_walls:
            return True
        else: 
            return False
    def has_east_wall(self):
        if self.wall_type in self.east_walls:
            return True
        else: 
            return False
    def has_south_wall(self):
        if self.wall_type in self.south_walls:
            return True
        else: 
            return False

"""
The Dungeon class gives us the size of the dungeon and its cells
As well as many utility functions
"""
class Dungeon:
    def __init__(self, width, height):

        self.width = width 
        self.height = height 
        self.ncells = self.width*self.height # number of cells in the dungeon
        
        # dictionary for cell wall type/configuration to string conversion
        # ██ are walls
        # :: are broken/open walls
        self.conversion = {
            0: """██::██
::  ::
██::██""", # no walls
            1: """██████
::  ::
██::██""", # single wall north
            2: """██::██
::  ██
██::██""", # single wall east
            3: """██::██
::  ::
██████""", # single wall south
            4: """██::██
██  ::
██::██""", # single wall west
            5: """██████
::  ██
██::██""", # double wall north + east
            6: """██::██
::  ██
██████""", # double wall east + south
            7: """██::██
██  ::
██████""", # double wall south + west
            8: """██████
██  ::
██::██""", # double wall west + north
            9: """██████
::  ::
██████""", # double wall north + south
            10: """██::██
██  ██
██::██""", # double wall west + east
            11: """██████
::  ██
██████""", # triple wall north + east + south
            12: """██::██
██  ██
██████""", # triple wall east + south + west
            13: """██████
██  ::
██████""", # triple wall south + west + north
            14: """██████
██  ██
██::██""", # triple wall west + north + east
            15: """██████
██  ██
██████""" # quadruple wall
        }
        
        # creates a 2d-array of cells representing our dungeon
        self.cells = [[Cell(np.random.choice(15)) for x in range(self.width)] for y in range(self.height)]
        
    
    """
    Prints a text representation of the dungeon
    Entrance is green, treasure is yellow, exit is red
    """
    def __repr__(self):
        # print multiline strings next to each other
        for y in range(self.height):
            lines = []
            for x in range(self.width):
                if self.cells[y][x].is_entrance:                    
                    lines.append(self.conversion[self.cells[y][x].wall_type].replace("  ","\033[0;32m██\033[0;0m").splitlines())
                elif self.cells[y][x].is_treasure:
                    lines.append(self.conversion[self.cells[y][x].wall_type].replace("  ","\033[0;33m██\033[0;0m").splitlines())
                elif self.cells[y][x].is_exit:
                    lines.append(self.conversion[self.cells[y][x].wall_type].replace("  ","\033[0;31m██\033[0;0m").splitlines())
                else:
                    lines.append(self.conversion[self.cells[y][x].wall_type].splitlines())            
            for line in zip(*lines):
                print(*line, sep='')
        return("")     
    

        
    """
    Returns a list of cells immediately (neighbours) connected to the one at (x,y) (with open walls)
    """  
    def find_connected_cells(self,x,y):
        connected_cells = []
        if y != 0:
            if (self.cells[x][y].has_west_wall() == False and self.cells[x][y-1].has_east_wall() == False): connected_cells.append([x,y-1])
        if y != 3:        
            if (self.cells[x][y].has_east_wall() == False and self.cells[x][y+1].has_west_wall() == False): connected_cells.append([x,y+1])
        if x != 3:
            if (self.cells[x][y].has_south_wall() == False and self.cells[x+1][y].has_north_wall() == False): connected_cells.append([x+1,y])
        if x != 0:
            if (self.cells[x][y].has_north_wall() == False and self.cells[x-1][y].has_south_wall() == False): connected_cells.append([x-1,y])
        return connected_cells
        
    
    """
    Finds and returns the longest existing path in a given dungeon, i.e. longest streak of connected cells
    """
    def find_longest_path(self):
        previous_longest_path = []
        longest_path = []
        mid_indices = [[1,1],[1,2],[2,1],[2,2]]
        # start on middle cells which have the best chance to be on the longest path
        while len(mid_indices) > 0:
            ind = mid_indices[0]
            mid_indices.remove(ind)
            current_longest_path = []
            neighbours = []
            current_longest_path.append(ind)            
            # from there, find neighbours of neighbours until none exist
            neighbours = self.find_connected_cells(ind[0],ind[1])
            while len(neighbours) > 0:
                neighbour = neighbours[0]
                current_longest_path.append(neighbour)
                neighbours.remove(neighbour)
                if neighbour in mid_indices: mid_indices.remove(neighbour)
                new_neighbours = self.find_connected_cells(neighbour[0],neighbour[1])
                neighbours += [ngb for ngb in new_neighbours if ngb not in current_longest_path] 
            # if we found a longer path than before, update it
            if len(current_longest_path) > len(previous_longest_path):
                previous_longest_path = list(current_longest_path) 
        [longest_path.append(path) for path in previous_longest_path if path not in longest_path]
        longest_path.sort()
        return longest_path
    
    
    """
    Sample three cells along the longest path of the dungeon (connected_cells) and make them keypoints
    """
    def add_key_points(self,connected_cells):        
        key_indices = random.sample(connected_cells,3)
        self.cells[key_indices[0][0]][key_indices[0][1]].is_entrance = True
        self.cells[key_indices[1][0]][key_indices[1][1]].is_treasure = True  
        self.cells[key_indices[2][0]][key_indices[2][1]].is_exit = True

In [3]:
# utility functions

"""
This function creates and returns the rewards array for the training process
Rewards are based on the conditions for a suitable dungeon (e.g. exterior walls closed, long path)
"""
def initialize_rewards(dungeon):
    # initialize every reward at -20
    rewards = np.full((dungeon.width, dungeon.height, dungeon.ncells), -20.)
    # reward for closing top left corner
    rewards[0][0][8], rewards [0][0][13], rewards[0][0][14] = 20,20,20 
    # rewards for closing bottom right corner
    rewards[-1][-1][6], rewards[-1][-1][11], rewards[-1][-1][12] = 20,20,20 
    # rewards for closing top right corner
    rewards[0][-1][5], rewards[0][-1][11], rewards[0][-1][14] = 20,20,20 
    # rewards for closing bottom left corner
    rewards[-1][0][7], rewards[-1][0][12], rewards[-1][0][13] = 20,20,20 
    # rewards for closing north walls
    for i in range(1,dungeon.width-1): 
        for j in dungeon.cells[0][0].north_walls:
            rewards[0][i][j] = 20
    # rewards for closing east walls
    for i in range(1,dungeon.height-1): 
        for j in dungeon.cells[0][0].east_walls:
            rewards[i][-1][j] = 20
    # rewards for closing south walls
    for i in range(1,dungeon.width-1): 
        for j in dungeon.cells[0][0].south_walls:
            rewards[-1][i][j] = 20
    # rewards for closing west walls
    for i in range(1,dungeon.height-1):
        for j in dungeon.cells[0][0].west_walls:
            rewards[i][0][j] = 20
    # rewards for leaving center rooms more open (to get long paths more often)
    # by giving them at most one wall
    for i in range (1,dungeon.height-1):
        for j in range(1,dungeon.width-1):
            rewards[i][j][0:5] = 10
    return rewards


"""
Get a random argmax from an array in case of tie
"""
def randargmax(arr,**kw):
    return np.argmax(np.random.random(arr.shape) * (arr==arr.max()), **kw)  


"""
Returns the indices corresponding to the n maximum values from a numpy array
"""
def largest_indices(arr, n):
    flat = arr.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    unraveled = np.unravel_index(indices, arr.shape)
    return [[unraveled[j][i] for j in range(len(arr.shape))] for i in range(n)]


# during training, the next action is chosen using an epsilon greedy algorithm
# this determines the percentage of actions that will be "greedy", i.e. of random exploration, 
# rather than the optimal one from current q_values
"""
The function returns a tuple (cell_x,cell_y,action_id) corresponding to the action the AI will do
"""
def pick_next_action(epsilon):
    if np.random.random() < epsilon:
        # epsilon test passed; pick most efficient action
        #return np.unravel_index(randargmax(q_values,axis=None),q_values.shape)
        return random.choice(largest_indices(q_values,5))
    else:
        # pick random integers for the tile to modify, and one for the action to perform
        return (np.random.randint(dungeon.width), np.random.randint(dungeon.height), np.random.randint(len(actions)))

    
"""
Like pick_next_action, but for a specific cell with (x,y) coordinates
Only returns the action id
"""
def pick_next_action_localized(x,y,epsilon):
    if np.random.random() < epsilon:
        return random.choice(largest_indices(q_values[x][y],3))[0]
    else:
        return random.choice(q_values[x][y])     


In [4]:
# here we train the AI
# a state is simply the state of the dungeon at a given iteration
# the AI is free to modify any cell of the dungeon using the actions; it is not located in space

# initialize training dungeon
dungeon = Dungeon(4,4)
# actions are changing a dungeon cell to one of the 16 wall configurations
# as in the conversion dictionary from the dungeon class
actions = np.arange(16)
# 3D array for the Q function, storing a value for each (x,y,action) triplet, initialized at zero everywhere
q_values = np.zeros((dungeon.width, dungeon.height, len(actions)))
# initialize rewards based on desired dungeon shape
rewards = initialize_rewards(dungeon)

# training parameters
epsilon = 0.1 # used in the epsilon greedy function to determine whether to explore or exploit; small epsilon favors exploration
discount_factor = 0.8 # weight for future rewards
learning_rate = 0.8 # weight for rate of q_value changes
episodes = 2000 # training iterations
    
"""
This functions trains the AI agent using the Q-learning algorithm
by picking an action at each episode (chosen by epsilon greedy algorithm)
then getting the reward, computing the temporal difference and updating the Q-values
"""
def train_agent(epsilon,discount_factor,learning_rate,episodes):
    for episode in range(episodes):
        # pick next action and cell to do said action
        result_tuple = pick_next_action(epsilon)
        cell_x, cell_y, action_id = result_tuple[0], result_tuple[1], result_tuple[2]
    
        # do the action on the chosen cell
        dungeon.cells[cell_x][cell_y].wall_type = action_id    
        
        # pick reward for the action and compute temporal difference
        reward = rewards[cell_x, cell_y, action_id]
        old_q_value = q_values[cell_x, cell_y, action_id]
        temporal_difference = reward + (discount_factor * np.max(q_values[cell_x, cell_y])) - old_q_value
    
        # Bellman equation to update q_values
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[cell_x, cell_y, action_id] = new_q_value
    
    print("Training complete!")

train_agent(epsilon,discount_factor,learning_rate,episodes)

Training complete!


In [5]:
# now that the algorithm is trained, we use it to alter a newly generated random dungeon
# by letting the IA do a full sweep of modifications, optimally picked from trained q_values, to the dungeon

# initialize a random dungeon and print it
new_dungeon = Dungeon(4,4)
print(new_dungeon)

"""
The function goes over every cell of a dungeon to make at most one change to each
The change to each cell is picked at random between best choices to ensure every dungeon is different
The resulting dungeon should have closed exterior walls and a big enough path (controlled by path_size_threshold)
The has_keypoints boolean is for opting in or out of keypoints. If True, the dungeon will have an entrance, 
a treasure, and an exit along the longest path
"""
def make_dungeon_suitable(dung,path_size_threshold,has_keypoints):
    path_length = 0 # the largest path of connected cells in the dungeon
    connected_cells = []
    while path_length < path_size_threshold: # keep only dungeons with at least path_size_threshold connected cells
        for i in range(dung.height):
            for j in range(dung.width):
                action_id = pick_next_action_localized(i,j,1)
                dung.cells[i][j].wall_type = action_id
        connected_cells = dung.find_longest_path()
        path_length = len(connected_cells)  
    # add key points randomly along longest path if keypoints opted in
    if has_keypoints:
       new_dungeon.add_key_points(connected_cells)
        

# let the AI agent act on the new dungeon and print it afterwards
# walls are black, :: are open walls, entrance is green, treasure is yellow, exit is red
make_dungeon_suitable(new_dungeon,12,True)
print(new_dungeon)

██::████::██████████::██
::  ██::  ██::  ████  ::
██::██████████::████████
██::████::████::████::██
██  ::::  ::::  ::██  ██
██::████::██████████::██
████████████████████::██
██  ::::  ::::  ::██  ::
████████::██████████::██
██████████████::████::██
::  ::::  ::::  ██::  ::
████████::██████████████

████████████████████████
██  ██::  ::::  ::::  ██
██::████::████::████::██
██::████::████::████::██
██  ██::  ::::  ████  ██
██::████::████::████████
██::████::████::████::██
██  ::::  ::████::::  ██
██::██████████::████::██
██::██████████::████::██
██  ::::██::::██::::  ██
████████████████████████

